#BaseLine Model

In [1]:
import gc
import pandas as pd
import numpy as np
import os

In [2]:
!gdown --id 196BS6Wq4gcL-JXWPFOryBidS9KC9CtJp
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions download -c instacart-market-basket-analysis

Downloading...
From: https://drive.google.com/uc?id=196BS6Wq4gcL-JXWPFOryBidS9KC9CtJp
To: /content/kaggle.json
100% 65.0/65.0 [00:00<00:00, 55.0kB/s]
 72% 5.00M/6.90M [00:00<00:00, 29.1MB/s]
100% 6.90M/6.90M [00:00<00:00, 33.8MB/s]
 97% 152M/157M [00:02<00:00, 56.7MB/s]
100% 157M/157M [00:02<00:00, 69.5MB/s]
  0% 0.00/1.87k [00:00<?, ?B/s]
100% 1.87k/1.87k [00:00<00:00, 1.64MB/s]
  0% 0.00/804 [00:00<?, ?B/s]
100% 804/804 [00:00<00:00, 696kB/s]
  0% 0.00/795k [00:00<?, ?B/s]
100% 795k/795k [00:00<00:00, 113MB/s]
  0% 0.00/220k [00:00<?, ?B/s]
100% 220k/220k [00:00<00:00, 71.0MB/s]
 29% 9.00M/31.3M [00:00<00:01, 22.2MB/s]
100% 31.3M/31.3M [00:00<00:00, 63.7MB/s]


In [3]:
!unzip orders.csv.zip
!unzip order_products__prior.csv.zip
!unzip order_products__train.csv.zip
!unzip aisles.csv.zip
!unzip departments.csv.zip
!unzip products.csv.zip

Archive:  orders.csv.zip
  inflating: orders.csv              
   creating: __MACOSX/
  inflating: __MACOSX/._orders.csv   
Archive:  order_products__prior.csv.zip
  inflating: order_products__prior.csv  
  inflating: __MACOSX/._order_products__prior.csv  
Archive:  order_products__train.csv.zip
  inflating: order_products__train.csv  
  inflating: __MACOSX/._order_products__train.csv  
Archive:  aisles.csv.zip
  inflating: aisles.csv              
  inflating: __MACOSX/._aisles.csv   
Archive:  departments.csv.zip
  inflating: departments.csv         
  inflating: __MACOSX/._departments.csv  
Archive:  products.csv.zip
  inflating: products.csv            
  inflating: __MACOSX/._products.csv  


In [5]:
!mkdir data
!cp drive/My\ Drive/eCloudvalley/chunk_0.pkl data/
!cp drive/My\ Drive/eCloudvalley/orders_comsum.pkl data/
!cp drive/My\ Drive/eCloudvalley/previous_products.pkl data/
!cp drive/My\ Drive/eCloudvalley/product_period.pkl data/
!cp drive/My\ Drive/eCloudvalley/product_periods_stat.pkl data/ 
!cp drive/My\ Drive/eCloudvalley/user_aisle_products.pkl data/
!cp drive/My\ Drive/eCloudvalley/user_department_products.pkl data/
!cp drive/My\ Drive/eCloudvalley/prod2vec.pkl data/

In [ ]:
!cp data/chunk_0.pkl drive/My\ Drive/eCloudvalley
!cp data/orders_comsum.pkl drive/My\ Drive/eCloudvalley
!cp data/previous_products.pkl drive/My\ Drive/eCloudvalley
!cp data/product_period.pkl drive/My\ Drive/eCloudvalley
!cp data/product_periods_stat.pkl drive/My\ Drive/eCloudvalley
!cp data/user_aisle_products.pkl drive/My\ Drive/eCloudvalley
!cp data/user_department_products.pkl drive/My\ Drive/eCloudvalley

In [4]:
path = "/content"
order_prior = pd.read_csv(os.path.join(path, "order_products__prior.csv"), dtype={'order_id': np.uint32,
                                           'product_id': np.uint16,
                                           'add_to_cart_order':np.uint8,
                                           'reordered': bool})

orders = pd.read_csv(os.path.join(path, "orders.csv"), dtype={'order_id':np.uint32,
                                 'user_id': np.uint32,
                                 'eval_set': 'category',
                                 'order_number':np.uint8,
                                 'order_dow': np.uint8,
                                 'order_hour_of_day': np.uint8
                                 })
print('loaded')

loaded


In [ ]:
orders = orders[orders['eval_set'] == 'prior']
orders_user = orders[['order_id', 'user_id']]
labels = pd.merge(order_prior, orders_user, on='order_id')
labels = labels[['user_id', 'product_id']].drop_duplicates()

print(labels)

print('save')
print(labels.shape)
print(labels.columns)
!mkdir data
labels.to_pickle('data/previous_products.pkl')

          user_id  product_id
0          202279       33120
1          202279       28985
2          202279        9327
3          202279       45918
4          202279       30035
...           ...         ...
32434480    25247       45309
32434481    25247       21162
32434483    25247       35211
32434485    25247       11352
32434486    25247        4600

[13307953 rows x 2 columns]
save
(13307953, 2)
Index(['user_id', 'product_id'], dtype='object')
mkdir: cannot create directory ‘data’: File exists


Split DataSet

In [6]:
folds = 1

aisles = pd.read_csv(os.path.join(path, "aisles.csv"), dtype={'aisle_id': np.uint8, 'aisle':'category'})
departments = pd.read_csv(os.path.join(path, "departments.csv"), dtype={'department_id':np.uint8, 'department': 'category'})
order_prior = pd.read_csv(os.path.join(path, "order_products__prior.csv"), dtype={'order_id': np.uint32,
                                           'product_id': np.uint16,
                                           'add_to_cart_order':np.uint8,
                                           'reordered': bool})
order_train = pd.read_csv(os.path.join(path, "order_products__train.csv"), dtype={'order_id': np.uint32,
                                           'product_id': np.uint16,
                                           'add_to_cart_order':np.uint8,
                                           'reordered': bool})
orders = pd.read_csv(os.path.join(path, "orders.csv"), dtype={'order_id':np.uint32,
                                 'user_id': np.uint32,
                                 'eval_set': 'category',
                                 'order_number':np.uint8,
                                 'order_dow': np.uint8,
                                 'order_hour_of_day': np.uint8})

products = pd.read_csv(os.path.join(path, "products.csv"), dtype={'product_id': np.uint16,
                                   'aisle_id': np.uint8,
                                   'department_id': np.uint8})

labels = pd.read_pickle('data/previous_products.pkl')

In [7]:
orders = orders[(orders.eval_set == 'train') | (orders.eval_set == 'test')]
labels = pd.merge(labels, orders[['order_id', 'user_id', 'eval_set']], on='user_id').drop(['user_id'], axis=1)
order_train.drop(['add_to_cart_order'], axis=1, inplace=True)

print('data is loaded')

orders = np.unique(labels.order_id)
size = orders.shape[0] // folds

data is loaded


In [19]:
order_train

,order_id,product_id,reordered
0,1,49302,True
1,1,11109,True
2,1,10246,False
3,1,49683,False
4,1,43633,True
...,...,...,...
1384612,3421063,14233,True
1384613,3421063,35548,True
1384614,3421070,35951,True
1384615,3421070,16953,True


In [14]:
labels.loc[np.in1d(labels.order_id, current), :]

,product_id,order_id,eval_set
0,33120,1050357,train
1,28985,1050357,train
2,9327,1050357,train
3,45918,1050357,train
4,30035,1050357,train
...,...,...,...
13307948,35221,2643210,test
13307949,21288,2643210,test
13307950,42615,2643210,test
13307951,25890,2643210,test


In [20]:
current = labels
current = pd.merge(order_train, current, on=['order_id', 'product_id'], how='right')
current.reordered.fillna(False, inplace=True)
current

,order_id,product_id,reordered,eval_set
0,1,49302,True,train
1,1,11109,True,train
2,1,43633,True,train
3,1,22035,True,train
4,36,19660,True,train
...,...,...,...,...
13307948,2643210,35221,False,test
13307949,2643210,21288,False,test
13307950,2643210,42615,False,test
13307951,2643210,25890,False,test


In [ ]:
for fold in range(folds):

    current = orders[fold * size:(fold + 1) * size]

    current = labels.loc[np.in1d(labels.order_id, current), :]

    current = pd.merge(order_train, current, on=['order_id', 'product_id'], how='right')
    current.reordered.fillna(False, inplace=True)
    print(current.columns)
    print(current.shape)

    current.to_pickle('data/chunk_{}.pkl'.format(fold))

Index(['order_id', 'product_id', 'reordered', 'eval_set'], dtype='object')
(13307953, 4)


In [ ]:
current

,order_id,product_id,reordered,eval_set
0,1,49302,True,train
1,1,11109,True,train
2,1,43633,True,train
3,1,22035,True,train
4,36,19660,True,train
...,...,...,...,...
13307948,2643210,35221,False,test
13307949,2643210,21288,False,test
13307950,2643210,42615,False,test
13307951,2643210,25890,False,test


In [ ]:
orders = pd.read_csv(os.path.join(path, "orders.csv"), dtype={'order_id':np.uint32,
                                 'user_id': np.uint32,
                                 'eval_set': 'category',
                                 'order_number':np.uint8,
                                 'order_dow': np.uint8,
                                 'order_hour_of_day': np.uint8})
labels = pd.read_pickle(os.path.join(path, 'data/chunk_0.pkl'))
user_product = pd.read_pickle(os.path.join(path, 'data/previous_products.pkl'))

order_comsum = orders[['user_id', 'order_number', 'days_since_prior_order']].groupby(['user_id', 'order_number'])\
            ['days_since_prior_order'].sum().groupby(level=[0]).cumsum().reset_index().rename(columns={'days_since_prior_order':'days_since_prior_order_comsum'})

# order_comsum['days_since_prior_order_comsum'].fillna(0, inplace=True)
order_comsum.to_pickle('data/orders_comsum.pkl')

In [ ]:
order_comsum

,user_id,order_number,days_since_prior_order_comsum
0,1,1,0.0
1,1,2,15.0
2,1,3,36.0
3,1,4,65.0
4,1,5,93.0
...,...,...,...
3421078,206209,10,155.0
3421079,206209,11,185.0
3421080,206209,12,203.0
3421081,206209,13,210.0


In [ ]:
order_comsum = pd.merge(order_comsum, orders, on=['user_id', 'order_number'])[['user_id', 'order_number', 'days_since_prior_order_comsum', 'order_id']]

order_product = pd.merge(order_prior, orders, on='order_id')[['order_id', 'product_id', 'eval_set']]
order_product_train_test = labels[['order_id', 'product_id', 'eval_set']]

order_product = pd.concat([order_product, order_product_train_test])

order_product = pd.merge(order_product, order_comsum, on='order_id')

print(order_product.columns)

Index(['order_id', 'product_id', 'eval_set', 'user_id', 'order_number',
       'days_since_prior_order_comsum'],
      dtype='object')


In [ ]:
order_product

,order_id,product_id,eval_set,user_id,order_number,days_since_prior_order_comsum
0,2,33120,prior,202279,3,28.0
1,2,28985,prior,202279,3,28.0
2,2,9327,prior,202279,3,28.0
3,2,45918,prior,202279,3,28.0
4,2,30035,prior,202279,3,28.0
...,...,...,...,...,...,...
45742437,2643210,35221,test,179441,4,65.0
45742438,2643210,21288,test,179441,4,65.0
45742439,2643210,42615,test,179441,4,65.0
45742440,2643210,25890,test,179441,4,65.0


In [ ]:
order_product = pd.merge(order_product, user_product, on=['user_id', 'product_id'])

temp = order_product.groupby(['user_id', 'product_id', 'order_number'])['days_since_prior_order_comsum'].sum().groupby(level=[0, 1]).apply(lambda x: np.diff(np.nan_to_num(x)))
temp = temp.to_frame('periods').reset_index()

temp.to_pickle('data/product_period.pkl')

In [ ]:
temp

,user_id,product_id,periods
0,1,196,"[15.0, 21.0, 29.0, 28.0, 19.0, 20.0, 14.0, 0.0..."
1,1,10258,"[21.0, 29.0, 28.0, 19.0, 20.0, 14.0, 0.0, 30.0..."
2,1,10326,[97.0]
3,1,12427,"[15.0, 21.0, 29.0, 28.0, 19.0, 20.0, 14.0, 0.0..."
4,1,13032,"[117.0, 44.0, 14.0]"
...,...,...,...
13307948,206209,43961,"[37.0, 99.0, 37.0]"
13307949,206209,44325,[158.0]
13307950,206209,48370,[55.0]
13307951,206209,48697,[158.0]


In [ ]:
aggregated = temp.copy()
aggregated['last'] = aggregated.periods.apply(lambda x: x[-1])
aggregated['prev1'] = aggregated.periods.apply(lambda x: x[-2] if len(x) > 1 else np.nan)
aggregated['prev2'] = aggregated.periods.apply(lambda x: x[-3] if len(x) > 2 else np.nan)
aggregated['median'] = aggregated.periods.apply(lambda x: np.median(x[:-1]))
aggregated['mean'] = aggregated.periods.apply(lambda x: np.mean(x[:-1]))
aggregated.drop('periods', axis=1, inplace=True)

aggregated.to_pickle('data/product_periods_stat.pkl')

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [ ]:
aggregated

,user_id,product_id,last,prev1,prev2,median,mean
0,1,196,14.0,30.0,0.0,20.0,19.555556
1,1,10258,14.0,30.0,0.0,20.5,20.125000
2,1,10326,97.0,NaN,NaN,NaN,NaN
3,1,12427,14.0,30.0,0.0,20.0,19.555556
4,1,13032,14.0,44.0,117.0,80.5,80.500000
...,...,...,...,...,...,...,...
13307948,206209,43961,37.0,99.0,37.0,68.0,68.000000
13307949,206209,44325,158.0,NaN,NaN,NaN,NaN
13307950,206209,48370,55.0,NaN,NaN,NaN,NaN
13307951,206209,48697,158.0,NaN,NaN,NaN,NaN


In [ ]:
order_train = pd.read_pickle(os.path.join(path, 'data/chunk_0.pkl'))

orders_products = pd.merge(orders, order_prior, on="order_id")

orders_products_products = pd.merge(orders_products, products[['product_id', 'department_id', 'aisle_id']],
                                    on='product_id')

user_dep_stat = orders_products_products.groupby(['user_id', 'department_id']).agg(
    {'product_id': lambda x: x.nunique(),
      'reordered': 'sum'
      })
print(user_dep_stat.columns)
user_dep_stat.rename(columns={'product_id': 'dep_products',
                              'reordered': 'dep_reordered'}, inplace=True)
user_dep_stat.reset_index(inplace=True)
print(user_dep_stat.columns)
user_dep_stat.to_pickle('data/user_department_products.pkl')

user_aisle_stat = orders_products_products.groupby(['user_id', 'aisle_id']).agg(
    {'product_id': lambda x: x.nunique(),
      'reordered': 'sum'
      })
print(user_aisle_stat.columns)
user_aisle_stat.rename(columns={'product_id': 'aisle_products',
                                'reordered': 'aisle_reordered'}, inplace=True)
user_aisle_stat.reset_index(inplace=True)
user_aisle_stat.to_pickle('data/user_aisle_products.pkl')

Index(['product_id', 'reordered'], dtype='object')
Index(['user_id', 'department_id', 'dep_products', 'dep_reordered'], dtype='object')
Index(['product_id', 'reordered'], dtype='object')


In [ ]:
pd.read_pickle('data/user_department_products.pkl')

,user_id,department_id,dep_products,dep_reordered
0,1,4,4,1.0
1,1,7,2,11.0
2,1,13,1,0.0
3,1,14,1,2.0
4,1,16,5,8.0
...,...,...,...,...
2232784,206209,15,4,3.0
2232785,206209,16,13,15.0
2232786,206209,17,8,6.0
2232787,206209,19,9,2.0


In [ ]:
pd.read_pickle('data/user_aisle_products.pkl')

,user_id,aisle_id,aisle_products,aisle_reordered
0,1,21,1,7.0
1,1,23,2,10.0
2,1,24,4,1.0
3,1,45,1,0.0
4,1,53,1,1.0
...,...,...,...,...
5729244,206209,120,3,0.0
5729245,206209,121,4,2.0
5729246,206209,123,3,2.0
5729247,206209,130,2,1.0


In [ ]:
def create_list(df):
    add_to_cart_order = df.add_to_cart_order.values
    values = df.product_id.values
    index = np.argsort(add_to_cart_order)
    values = values[index].tolist()
    return values

In [ ]:
path = '/content'
order_prior = pd.read_csv(os.path.join(path, "order_products__prior.csv"), dtype={'order_id': np.uint32,
                                           'product_id': np.uint16,
                                           'add_to_cart_order': np.uint8})
orders = pd.read_csv(os.path.join(path, "orders.csv"), dtype={'order_id': np.uint32,
                                 'order_dow': np.uint8,
                                 'order_hour_of_day': np.uint8})

data = pd.merge(order_prior, orders, on='order_id')

data = order_prior.sort_values(['order_id']).groupby('order_id')['product_id']\
    .apply(lambda x: x.tolist()).to_frame('products').reset_index()
data = pd.merge(data, orders, on='order_id')
data.to_pickle(os.path.join(path, 'prod2vec.pkl'))

In [ ]:
!cp drive/My\ Drive/eCloudvalley/prod2vec.pkl data/

In [ ]:
data

,order_id,products,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,"[33120, 28985, 9327, 45918, 30035, 17794, 4014...",202279,prior,3,5,9,8.0
1,3,"[32665, 17461, 46667, 17668, 17704, 24838, 337...",205970,prior,16,5,17,12.0
2,4,"[40285, 41276, 32645, 25146, 17616, 34862, 100...",178520,prior,36,1,9,7.0
3,5,"[12962, 45698, 24773, 18569, 41176, 46522, 472...",156122,prior,42,6,16,9.0
4,6,"[41897, 40462, 15873]",22352,prior,4,1,12,30.0
...,...,...,...,...,...,...,...,...
3214869,3421079,[30136],108687,prior,6,1,13,0.0
3214870,3421080,"[38061, 10667, 25122, 12935, 41950, 18811, 493...",52726,prior,2,1,11,2.0
3214871,3421081,"[12861, 35221, 20539, 12218, 32299, 38185, 3060]",117076,prior,1,3,11,NaN
3214872,3421082,"[17279, 12738, 16797, 43352, 32700, 12023, 47941]",175185,prior,23,2,18,4.0


In [ ]:
import warnings
from concurrent.futures import ThreadPoolExecutor
from functools import reduce

import tensorflow as tf
import numpy as np
import math
from sklearn.utils import shuffle


class Product2VecSkipGram:
    def __init__(self, data, cv_data, batch_size, num_skips, skip_window, vocabulary_size, embedding_size=32,
                 num_negative_sampled=64, len_ratio = 0.5):
        self.data = data
        self.cv_data = cv_data
        self.data_index = 0
        self.batch_size = batch_size
        self.num_skips = num_skips
        self.skip_window = skip_window
        self.embedding_size = embedding_size
        self.num_negative_sampled = num_negative_sampled
        self.vocabulary_size = vocabulary_size
        self.len_ratio = len_ratio
        assert batch_size % num_skips == 0
        assert num_skips <= 2 * skip_window
        self.build_graph()

    def predict(self, products):
        result = []
        for i in range(0, len(products), self.batch_size):
            batch = products[i:i+self.batch_size]
            batch = self.sess.run(self.gathered, feed_dict={self.train_inputs: batch})
            result.append(batch)
        return np.concatenate(result, axis=0)

    def train(self, num_steps, cv_every_n_steps, cv_steps, lrs):
        with ThreadPoolExecutor(max_workers=2) as executor:
            average_loss = 0
            learning_rate = 1.0
            current = executor.submit(self.generate_batch)
            for step in range(num_steps):
                if step in lrs:
                    learning_rate = lrs[step]
                batch_inputs, batch_labels = current.result()
                current = executor.submit(self.generate_batch)
                feed_dict = {self.train_inputs: batch_inputs,
                             self.train_labels: batch_labels,
                             self.learning_rate: learning_rate}

                _, loss_val = self.sess.run([self.optimizer, self.loss], feed_dict=feed_dict)
                average_loss += loss_val

                if step % 2000 == 0:
                    if step > 0:
                        average_loss /= 2000
                    print('Average loss at step ', step, ': ', average_loss)
                    average_loss = 0
                if step % cv_every_n_steps == 0:
                    self.data = shuffle(self.data, random_state=0)
                    self.save_model(step)
                    cv_loss = 0
                    for batch_inputs, batch_labels in self.generate_test(cv_steps):
                        feed_dict = {self.train_inputs: batch_inputs,
                                     self.train_labels: batch_labels,
                                     self.learning_rate: learning_rate}
                        loss_val = self.sess.run(self.loss, feed_dict=feed_dict)
                        cv_loss += loss_val
                    print('CV',cv_loss / cv_steps)

    def save_model(self, step):
        self.saver.save(self.sess, 'models/prod2vec_skip_gram', global_step=step)

    def load_model(self, path):
        self.saver.restore(self.sess, path)

    def build_graph(self):
        self.train_inputs = tf.placeholder(tf.int32, shape=[self.batch_size])
        self.train_labels = tf.placeholder(tf.int32, shape=[self.batch_size])
        self.learning_rate = tf.placeholder(tf.float32)

        # variables
        embeddings = tf.Variable(tf.random_uniform([self.vocabulary_size, self.embedding_size], -1.0, 1.0))

        softmax_weights = tf.Variable(tf.truncated_normal([self.embedding_size, self.vocabulary_size],
                                                          stddev=1.0 / math.sqrt(self.embedding_size)))
        softmax_biases = tf.Variable(tf.zeros([self.vocabulary_size]))

        self.gathered = tf.gather(embeddings, self.train_inputs)

        prediction = tf.matmul(self.gathered, softmax_weights) + softmax_biases
        self.loss = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(labels=self.train_labels, logits=prediction))

        self.optimizer = tf.train.GradientDescentOptimizer(self.learning_rate).minimize(self.loss)

        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        self.saver = tf.train.Saver()


    def inc(self):
        self.data_index = (self.data_index + 1) % len(self.data)

    def inc_cv(self, data_index):
        return (data_index + 1) % len(self.cv_data)

    def generate_batch(self):
        batch = np.ndarray(shape=(self.batch_size), dtype=np.int32)
        labels = np.ndarray(shape=(self.batch_size), dtype=np.int32)
        counter = 0
        while counter < self.batch_size:
            current = self.data.iloc[self.data_index]
            if len(current) == 1:
                warnings.warn("lenght is one", RuntimeWarning)
                self.inc()
                continue

            span = min(2 * self.skip_window + 1, len(current))

            x = target = np.random.randint(0, len(current))

            targets_to_avoid = [x]

            for j in range(self.num_skips):  # target varies!!! X constant!
                while target in targets_to_avoid and len(targets_to_avoid) != span:
                    target = np.random.randint(0, span)
                if len(targets_to_avoid) == span or counter == self.batch_size:
                    break
                targets_to_avoid.append(target)
                batch[counter] = current[x]
                labels[counter] = current[target]
                counter += 1
            self.inc()

        return batch, labels

    def generate_test(self, num_steps):
        data_index = 0
        for _ in range(num_steps):
            batch = np.ndarray(shape=(self.batch_size), dtype=np.int32)
            labels = np.ndarray(shape=(self.batch_size), dtype=np.int32)

            counter = 0
            while counter < self.batch_size:
                current = self.cv_data.iloc[data_index]
                if len(current) == 1:
                    warnings.warn("lenght is one", RuntimeWarning)
                    data_index = self.inc_cv(data_index)
                    continue

                span = min(2 * self.skip_window + 1, len(current))

                x = target = np.random.randint(0, len(current))

                targets_to_avoid = [x]

                for j in range(self.num_skips):  # target varies!!! X constant!
                    while target in targets_to_avoid and len(targets_to_avoid) != span:
                        target = np.random.randint(0, span)
                    if len(targets_to_avoid) == span or counter == self.batch_size:
                        break
                    targets_to_avoid.append(target)
                    batch[counter] = current[x]
                    labels[counter] = current[target]
                    counter += 1
                data_index = self.inc_cv(data_index)

            yield batch, labels


In [ ]:
np.random.seed(2017)
products = pd.read_csv('products.csv')
df = pd.read_pickle('data/prod2vec.pkl').products
print('initial size', len(df))

initial size 3214874


In [ ]:
from tqdm.notebook import trange
for i in trange(len(df.index)):
  if str(5273) in df[i]:
    print(i)


KeyboardInterrupt: ignored

In [ ]:
df[398806]

[42633,
 27323,
 1288,
 24184,
 16797,
 31683,
 44560,
 21497,
 10892,
 33651,
 38109,
 22825]

In [ ]:
def ListInttoStr(x):
  for i in range(len(x)):
    x[i] = str(x[i])
  return x

In [ ]:
df.apply(lambda x: ListInttoStr(x))

0          [33120, 28985, 9327, 45918, 30035, 17794, 4014...
1          [32665, 17461, 46667, 17668, 17704, 24838, 337...
2          [40285, 41276, 32645, 25146, 17616, 34862, 100...
3          [12962, 45698, 24773, 18569, 41176, 46522, 472...
4                                      [41897, 40462, 15873]
                                 ...                        
3214869                                              [30136]
3214870    [38061, 10667, 25122, 12935, 41950, 18811, 493...
3214871     [12861, 35221, 20539, 12218, 32299, 38185, 3060]
3214872    [17279, 12738, 16797, 43352, 32700, 12023, 47941]
3214873    [39678, 4600, 11352, 35211, 24852, 21162, 4530...
Name: products, Length: 3214874, dtype: object

In [ ]:
df[0]

['33120', '28985', '9327', '45918', '30035', '17794', '40141', '1819', '43668']

In [ ]:
from gensim.models import Word2Vec 

# train word2vec model
model2 = Word2Vec(window = 6, sg = 1, hs = 0,
                 negative = 64, # for negative sampling
                 alpha=0.5, min_alpha=0.1,
                 seed = 2020, min_count=1, workers=2, size=32)

model2.build_vocab(df, progress_per=200)

#model2.train(df, total_examples = model2.corpus_count, start_alpha=0.5, end_alpha=0.1, epochs=20, report_delay=1, compute_loss=True)

In [ ]:
len(model2.wv.vocab)

49677

In [ ]:
for i in range(49688):
  if str(i) not in model2.wv.vocab:
    print(i)

0
1002
1908
2769
3117
3426
3630
3718
3831
4417
4558
4626
4908
5081
5255
6062
6320
6681
6786
7045
7198
7626
7712
8435
9342
9818
10260
10270
10584
10806
11099
11254
11356
12372
12718
12823
13397
14020
14128
14485
14607
14756
15388
15877
15901
16461
16639
16790
17585
18001
18538
19259
20223
20264
20759
21428
21440
21483
22747
23624
23653
23741
24348
24402
25248
25383
25441
25482
25888
26117
26268
26627
27499
27512
28486
28504
28818
30068
30087
30149
30343
30451
31254
31333
31465
32021
32126
32464
33097
33421
33440
34211
34331
34463
34785
35542
35890
35952
36079
36233
36998
37014
37088
37660
37703
37981
38045
38276
38977
39522
39759
40744
41800
41967
42206
42235
42464
43098
43144
43725
43778
44495
44748
44982
44986
45273
45378
45380
45400
45893
45971
46278
46379
46625
46741
47408
47680
48151
48211
48343
48461
49487
49540


In [ ]:
model2['1288']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


KeyError: ignored

In [ ]:
print(model2)
# extract all vectors
X = model2[model2.wv.vocab]

X.shape

Word2Vec(vocab=49222, size=32, alpha=0.5)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


(49222, 32)

In [ ]:
products = pd.concat([products, pd.DataFrame(X)], axis=1)
products.to_pickle('data/product_embeddings.pkl')
!cp data/product_embeddings.pkl drive/My\ Drive/eCloudvalley

In [ ]:
model2.wv['33']

KeyError: ignored

In [ ]:
embd = model2(products.product_id.values)
products = pd.concat([products, pd.DataFrame(embd)], axis=1)
products.to_pickle('data/product_embeddings.pkl')

In [ ]:
np.random.seed(2017)
products = pd.read_csv('products.csv')
df = pd.read_pickle('data/prod2vec.pkl').products
print('initial size', len(df))

df_train, df_cv = train_test_split(df, test_size=0.1, random_state=2017)
batch_size = 1024
rates = {100000: 0.5,
          200000: 0.25,
          500000: 0.1}
model = Product2VecSkipGram(df_train, df_cv, batch_size, 1, 1, np.max(products.product_id) + 1)
model.train(120001, 20000, len(df_cv) // batch_size, rates)

initial size 3214874


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:114: RuntimeWarning: lenght is one


Average loss at step  0 :  10.93769645690918


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:147: RuntimeWarning: lenght is one


CV 10.940734561639852
Average loss at step  2000 :  10.262326110363007
Average loss at step  4000 :  9.62776603603363
Average loss at step  6000 :  9.33982783317566
Average loss at step  8000 :  9.16552994775772
Average loss at step  10000 :  9.042779479980469
Average loss at step  12000 :  8.951890106201171
Average loss at step  14000 :  8.881575417041779
Average loss at step  16000 :  8.826946051597595
Average loss at step  18000 :  8.781688512802123
Average loss at step  20000 :  8.745043810367584
CV 8.74127090624727
Average loss at step  22000 :  8.713645143985747
Average loss at step  24000 :  8.682539996147156
Average loss at step  26000 :  8.658969731807709
Average loss at step  28000 :  8.639849307060242
Average loss at step  30000 :  8.617630747795104
Average loss at step  32000 :  8.599885525226593
Average loss at step  34000 :  8.585291381835937
Average loss at step  36000 :  8.57153001832962
Average loss at step  38000 :  8.556509851455688
Average loss at step  40000 :  8.5

In [ ]:
#batch_size = 128
# rates = {100000: 0.5,
#           200000: 0.25,
#           500000: 0.1}
# model3 = Product2VecSkipGram(df_train, df_cv, len(products), 1, 1, np.max(products.product_id) + 1)
# model.load_model('models/prod2vec_skip_gram-120000')
embd = model.predict(products.product_id.values)
products = pd.concat([products, pd.DataFrame(embd)], axis=1)
products.to_pickle('data/product_embeddings.pkl')

ValueError: ignored

In [ ]:
embd = model.predict(products.product_id.values)
products = pd.concat([products, pd.DataFrame(embd)], axis=1)
products.to_pickle('data/product_embeddings.pkl')
!cp data/product_embeddings.pkl drive/My\ Drive/eCloudvalley

In [ ]:
embd

NameError: ignored

In [ ]:
products

,product_id,product_name,aisle_id,department_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,1,Chocolate Sandwich Cookies,61,19,0.022767,-0.512297,0.006285,0.158527,-0.080938,0.129847,0.256462,0.440197,0.316199,-0.106368,0.730227,-0.116449,0.309511,-0.269867,0.764530,-1.110666,-0.737164,-0.276994,0.237756,0.101570,-0.093639,0.025329,-1.377610,-0.680689,-0.517951,-0.885606,0.382417,-0.856607,0.749205,-0.771110,0.346901,-0.175627
1,2,All-Seasons Salt,104,13,0.080421,-0.744554,0.047977,0.733221,0.314641,0.217599,0.284836,-0.181505,-0.850737,-0.480058,-0.542350,0.853339,0.668314,0.156671,-0.110871,-0.461800,-0.740731,0.892361,0.349090,0.493269,0.508705,1.020658,0.781901,0.095789,-0.915559,-0.373763,-0.761947,0.353778,0.784436,-0.855943,0.277941,-0.160423
2,3,Robust Golden Unsweetened Oolong Tea,94,7,0.198683,-0.222948,-0.808351,-0.034940,0.620032,0.952705,-0.582246,-0.566011,0.808398,0.672292,-0.655767,-0.465853,0.555404,0.275444,-0.312616,-0.355634,-0.268104,0.600117,-0.752465,-0.117714,-0.276577,0.747497,-0.501080,-0.572618,-0.719068,0.518022,-0.762005,0.848908,0.449306,-0.469338,0.122560,-0.694554
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,0.484485,-0.510683,-0.410571,-0.564425,0.099906,0.714246,0.757104,-0.277754,0.395709,0.229250,-0.511095,-0.440126,0.354504,-0.390228,-0.016356,0.689560,0.678561,0.489012,0.656766,0.386529,-0.566478,0.419739,-0.366888,-0.166469,0.488966,0.295622,-0.119693,0.324078,0.055535,-0.512483,0.049819,0.030913
4,5,Green Chile Anytime Sauce,5,13,0.102635,0.919654,-0.598482,-0.575494,-0.583376,0.524451,-0.076345,0.158248,-0.894048,-0.917328,-0.262640,0.695169,0.734669,0.673996,0.800981,-0.966089,-0.916609,0.910535,-0.123362,-0.851140,0.873812,-0.300754,-0.761012,-0.474648,-0.147471,-0.383463,-0.551527,-0.643020,-0.300493,0.208861,0.302752,0.798204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5,-0.847023,-0.222026,0.318286,0.750334,0.789836,-0.561919,-0.212524,-0.863824,-0.798281,0.670478,0.417401,0.485525,0.089181,-0.784919,-0.378955,0.760471,0.879928,-0.760732,-0.884525,-0.093537,0.250918,0.785239,0.498732,-0.137281,-0.881301,0.234210,0.305039,-0.703012,-0.654087,0.643868,-0.511000,-0.075759
49684,49685,En Croute Roast Hazelnut Cranberry,42,1,-0.699170,0.120518,0.185956,-0.434755,-0.425873,-0.784608,-0.930097,-0.102469,0.748431,0.119191,0.688636,-0.199132,-0.768906,0.551132,-0.783905,-0.915948,-0.818223,0.328199,-0.671587,-0.754397,-0.145861,0.638060,0.493439,-0.884608,0.686051,-0.425731,-0.186972,0.097391,-0.190785,0.881174,-0.336932,-0.695090
49685,49686,Artisan Baguette,112,3,-0.884306,0.609275,-0.120718,-0.752040,-0.260331,0.075539,-0.506204,0.435785,-0.129692,-0.177639,0.228714,-0.860868,-0.372268,-0.178082,-0.613786,0.608416,0.792370,-0.561864,0.257614,-0.850175,-0.324410,0.813794,-0.681819,0.674616,0.178967,0.318419,-0.829099,0.307698,0.092161,0.983885,0.920722,-0.051931
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8,0.630068,-0.471400,0.581300,0.116848,0.841123,0.926723,-0.816249,-0.664044,0.651881,-0.020876,0.338523,0.142862,-0.957063,-0.518162,0.115253,-0.527973,-0.023630,-0.025538,-0.022698,-0.189851,0.297278,0.101500,0.625052,0.797530,0.567572,0.194094,0.402171,-0.755601,-0.633145,-0.717179,-0.902466,-0.185621
